# Project

In [1]:
import os
import pytimeloop.timeloopfe.v4 as tl

In [21]:
def run_timeloop_model(architecture, mapping, problem):
    if os.path.exists("./output_dir"):
        os.system("rm -r ./output_dir")

    spec = tl.Specification.from_yaml_files(architecture, mapping, problem)
    
    return tl.call_model(spec, output_dir="./output_dir")

### Data-Parallel

In [30]:
result = run_timeloop_model("./designs/arch.yaml", "./designs/map_dp.yaml", "./designs/prob.yaml")
stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()
print(stats)

Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== MAC ===

    SPECS
    -----
    Word bits             : 8
    Instances             : 16 (16*1)
    Compute energy        : 0.84 pJ

    STATS
    -----
    Utilized instances      : 16
    Computes (total)        : 14332723200
    Cycles                  : 895795200
    Energy (total)          : 12111151104.00 pJ
    Area (total)            : 7462.00 um^2

Level 1
-------
=== LocalBuffer ===

    SPECS
    -----
        Technology                      : SRAM
        Size                            : 32768
        Word bits                       : 8
        Block size                      : 8
        Cluster size                    : 1
        Instances                       : 16 (16*1)
        Shared bandwidth                : -
        Read bandwidth                  : -
        Write bandwidth                 : -
        Multiple buffering              : 1.00
        Effective size                  : 3276